In [2]:
import os
import sys
sys.path.append('/home/hkhachatrian/mae/')
import cv2
import numpy as np
from tqdm import tqdm_notebook as tqdm
import torch
from torch import nn

from get_model import get_model, get_model_config
from modeling import Encoder


In [6]:
import pickle

with open(f'results_dino_224_cos.pkl', 'rb') as f:
    results = pickle.load(f)
results

defaultdict(list,
            {'bbox_results': [[array([[173.62692   , 114.52495   , 183.35635   , 139.01265   ,
                         0.5645227 ],
                      [108.67286   , 108.37143   , 120.85171   , 129.90848   ,
                         0.36227456],
                      [157.68205   , 113.41237   , 169.90196   , 141.06651   ,
                         0.3162076 ]], dtype=float32),
               array([], shape=(0, 5), dtype=float32),
               array([[1.44274246e+02, 1.13806526e+02, 1.71140457e+02, 1.38688599e+02,
                       9.75703716e-01],
                      [1.46509056e+01, 1.09626625e+02, 3.90013123e+01, 1.30751770e+02,
                       9.56458092e-01],
                      [1.30697464e+02, 1.12358902e+02, 1.46342361e+02, 1.29629517e+02,
                       6.94400728e-01],
                      [1.39409592e+02, 1.13350311e+02, 1.61142471e+02, 1.34406036e+02,
                       5.59647739e-01],
                      [1.36167160e+

In [5]:
results['track_results']

[[array([], shape=(0, 6), dtype=float64),
  array([], shape=(0, 6), dtype=float64),
  array([[  0.        , 144.27424622, 113.80652618, 171.14045715,
          138.68859863,   0.97570372],
         [  1.        ,  14.65090561, 109.62662506,  39.00131226,
          130.75177002,   0.95645809]]),
  array([], shape=(0, 6), dtype=float64),
  array([], shape=(0, 6), dtype=float64),
  array([], shape=(0, 6), dtype=float64),
  array([], shape=(0, 6), dtype=float64),
  array([], shape=(0, 6), dtype=float64)],
 [array([], shape=(0, 6), dtype=float64),
  array([], shape=(0, 6), dtype=float64),
  array([[  0.        , 144.28681946, 114.0351181 , 171.89839172,
          138.66748047,   0.99324942],
         [  1.        ,  13.50999641, 109.90431213,  38.35920715,
          130.97773743,   0.90842175]]),
  array([], shape=(0, 6), dtype=float64),
  array([], shape=(0, 6), dtype=float64),
  array([], shape=(0, 6), dtype=float64),
  array([], shape=(0, 6), dtype=float64),
  array([], shape=(0, 6), dty

In [8]:
def image_normalize(image):
    imagenet_mean = np.array([0.485, 0.456, 0.406])
    imagenet_std = np.array([0.229, 0.224, 0.225])
    image = image / 255.
    image = image - imagenet_mean
    image = image / imagenet_std
    return image

In [3]:
import math
math.ceil(3/4)

1

In [ ]:
def get_embeds(model_name, annot_file):

    annot_file = np.load(annot_file, allow_pickle=True).item()
    
    encoder = Encoder(model_name)
    encoder.eval()

    for img in tqdm(annot_file['images']):
            labels = []
            embeddings = []
            if key == '_no_shift':
                PATH='./images/f1m/224_8shot/train/'
                saving_path = f'./embeddings/fair1m/{model}_224_8shot'


            path = os.path.join(PATH, img['file_name'])
            image = cv2.imread(path)
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

            proc_image = image_normalize(image)
            proc_image = torch.from_numpy(proc_image).unsqueeze(0)
            proc_image = torch.einsum('nhwc->nchw', proc_image)

            with torch.no_grad():
                img_enc = encoder.forward(proc_image.to('cuda'))
            
            embeddings.append(img_enc.cpu().detach().squeeze(0).numpy())
            if model == 'dinov2':
                if key == '_no_shift':
                    labels.append(img['patch_labels_16x16'])
                else:
                    labels.append(img[f'patch_labels_16x16_{key[-1]}'])
            else:
                if key == '_no_shift':
                    labels.append(img['patch_labels'])
                else:
                    labels.append(img[f'patch_labels_{key[-1]}'])

            embeddings = np.concatanate(embeddings, axis=0)
            labels = np.concatanate(labels, axis=0)

            print(embeddings.shape, labels.shape)

            print("saving embeds path:", saving_path+'_embeddings.npy')

            np.save(saving_path+'_embeddings.npy', embeddings)
            np.save(saving_path+'_labels.npy', embeddings)



embeds = get_embeds('dino', './annotations/f1m/250_8shot_val.npy')

In [3]:
a = np.load('/mnt/lwll/lwll-coral/hrant/mae_checkpoints/tracking/bdd100k/embeds_mae_224/track/val/b1c9c847-3bda4659/b1c9c847-3bda4659-0000001.npy')
a.shape

(196, 768)

In [4]:
x = torch.zeros((1,4), device='cpu')
x[:1].shape

torch.Size([1, 4])

In [4]:
import json
with open('/mnt/lwll/lwll-coral/hrant/mae_checkpoints/tracking/bdd/labels_dinov2_224/box_track_20/box_track_val_cocofmt.json') as f:
    label_data = json.load(f)

label_data

{'categories': [{'id': 1, 'name': 'pedestrian'},
  {'id': 2, 'name': 'rider'},
  {'id': 3, 'name': 'car'},
  {'id': 4, 'name': 'truck'},
  {'id': 5, 'name': 'bus'},
  {'id': 6, 'name': 'train'},
  {'id': 7, 'name': 'motorcycle'},
  {'id': 8, 'name': 'bicycle'}],
 'videos': [{'id': 1, 'name': 'b1c66a42-6f7d68ca', 'attributes': None},
  {'id': 2, 'name': 'b1c81faa-3df17267', 'attributes': None},
  {'id': 3, 'name': 'b1c81faa-c80764c5', 'attributes': None},
  {'id': 4, 'name': 'b1c9c847-3bda4659', 'attributes': None},
  {'id': 5, 'name': 'b1ca2e5d-84cf9134', 'attributes': None},
  {'id': 6, 'name': 'b1cac6a7-04e33135', 'attributes': None},
  {'id': 7, 'name': 'b1cd1e94-549d0bfe', 'attributes': None},
  {'id': 8, 'name': 'b1ceb32e-3f481b43', 'attributes': None},
  {'id': 9, 'name': 'b1ceb32e-51852abe', 'attributes': None},
  {'id': 10, 'name': 'b1cebfb7-284f5117', 'attributes': None},
  {'id': 11, 'name': 'b1d0091f-75824d0d', 'attributes': None},
  {'id': 12, 'name': 'b1d0091f-f2c2d2ae', '

In [ ]:
label_data['']

In [1]:
from mmdet.models import HEADS, build_head, build_roi_extractor

In [2]:
cfg = dict(
            type='SingleRoIExtractor',
            roi_layer=dict(type='RoIAlign', output_size=7, sampling_ratio=0),
            out_channels=256,
            featmap_strides=[16])


m = build_roi_extractor(cfg)

In [3]:
m

SingleRoIExtractor(
  (roi_layers): ModuleList(
    (0): RoIAlign(output_size=(7, 7), spatial_scale=0.0625, sampling_ratio=0, pool_mode=avg, aligned=True, use_torchvision=False)
  )
)

In [9]:
import torch
import numpy as np
b = m(torch.from_numpy(np.zeros((5,100,100))), torch.tensor([[0, 0.1,0.1,0.70,0.70]]))

IndexError: Dimension out of range (expected to be in range of [-2, 1], but got 2)